In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format='retina'

### Loading & Exploring Data

In [7]:
ipums = pd.read_csv('data/IPUMS_2019.csv')

In [8]:
ipums.columns

Index(['YEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'CPI99',
       'STATEFIP', 'COUNTYFIP', 'CITY', 'STRATA', 'GQ', 'OWNERSHP',
       'OWNERSHPD', 'OWNCOST', 'RENTGRS', 'HHINCOME', 'CILAPTOP', 'CIHISPEED',
       'PERNUM', 'PERWT', 'FAMSIZE', 'AGE', 'RACE', 'RACED', 'HISPAN',
       'HISPAND', 'EDUC', 'EDUCD', 'OCCSOC', 'INDNAICS', 'INCTOT', 'FTOTINC',
       'INCWAGE', 'MIGCOUNTY1', 'MOVEDIN', 'TRANTIME'],
      dtype='object')

In [9]:
ipums_eda = ipums[['SERIAL', 'SAMPLE', 'HHWT', 'HHINCOME', 'PERNUM', 'PERWT', 'FAMSIZE', 'INCTOT', 'FTOTINC', 'INCWAGE']]

In [10]:
ipums_eda.head(20)

,SERIAL,SAMPLE,HHWT,HHINCOME,PERNUM,PERWT,FAMSIZE,INCTOT,FTOTINC,INCWAGE
0,1,201901,11.0,9999999,1,11.0,1,9000,9999999,0
1,2,201901,70.0,9999999,1,70.0,1,150,9999999,0
2,3,201901,20.0,9999999,1,20.0,1,1400,9999999,1400
3,4,201901,79.0,9999999,1,79.0,1,22700,9999999,0
4,5,201901,53.0,9999999,1,53.0,1,0,9999999,0
5,6,201901,77.0,9999999,1,77.0,1,0,9999999,0
6,7,201901,8.0,9999999,1,8.0,1,36000,9999999,0
7,8,201901,15.0,9999999,1,15.0,1,9300,9999999,0
8,9,201901,61.0,9999999,1,61.0,1,60000,9999999,60000
9,10,201901,152.0,9999999,1,152.0,1,0,9999999,0


In [11]:
print(len(ipums_eda['FAMSIZE'].unique()), ipums_eda['FAMSIZE'].unique())

20 [ 1  2  4  3  6  5  8  7 12 10  9 11 13 14 15 19 16 20 18 17]


In [12]:
ipums_eda.loc[~(ipums_eda['PERWT'] == ipums_eda['HHWT'])]

,SERIAL,SAMPLE,HHWT,HHINCOME,PERNUM,PERWT,FAMSIZE,INCTOT,FTOTINC,INCWAGE
2387,2388,201901,370.0,297000,1,369.0,2,146000,297000,146000
2388,2388,201901,370.0,297000,2,425.0,2,151000,297000,109000
2389,2389,201901,94.0,16700,1,95.0,1,16700,16700,0
2390,2390,201901,144.0,64000,1,145.0,4,16500,64000,0
2391,2390,201901,144.0,64000,2,204.0,4,34500,64000,15600
...,...,...,...,...,...,...,...,...,...,...
3239547,1428034,201901,56.0,74400,2,72.0,2,16700,74400,18400
3239548,1428035,201901,29.0,131000,1,30.0,2,117000,131000,102000
3239549,1428035,201901,29.0,131000,2,123.0,2,14000,131000,14000
3239550,1428036,201901,51.0,18500,1,50.0,1,18500,18500,0


In [13]:
ipums_titles = pd.read_csv('data/ind_indnaics_crosswalk_2000_onward_without_code_descriptions.csv')

In [14]:
ipums_titles = ipums_titles.iloc[2:]

In [15]:
ipums_titles = ipums_titles.iloc[:,10:]

In [16]:
county_info = pd.read_csv('data/county_to_regions_key - Sheet1.csv')

### Cleaning

In [17]:
ipums = ipums[['STATEFIP', 
               'COUNTYFIP',
              'INDNAICS',
              'PERWT',
              'INCWAGE']]

In [18]:
ca_ipums = ipums.loc[ipums['STATEFIP'] == 6].copy()

In [19]:
ca_ipums = ca_ipums.reset_index()

In [20]:
len(ca_ipums)

380091

In [21]:
def normalize_titles(col):
    col = col.astype(str)
    col = col.str.strip()
    col = col.str.lower()
    col = col.str.replace('&', 'and')
    col = col.apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
    return col

In [22]:
ca_ipums['INDNAICS'] = normalize_titles(ca_ipums['INDNAICS'])

In [23]:
ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'] = normalize_titles(ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'])

### Merging on NAICS Codes

In [24]:
ca_ipums.columns.values

array(['index', 'STATEFIP', 'COUNTYFIP', 'INDNAICS', 'PERWT', 'INCWAGE'],
      dtype=object)

In [25]:
ipums_titles.columns.values

array(['2018 Onward ACS/PRCS INDNAICS CODE', 'Industry Title'],
      dtype=object)

In [26]:
ipums_merged = pd.merge(ca_ipums, ipums_titles,
                              left_on = 'INDNAICS', right_on = '2018 Onward ACS/PRCS INDNAICS CODE')

In [27]:
print(len(ca_ipums), len(ipums_titles), ipums_merged.shape)

380091 402 (224526, 8)


In [28]:
print(f'''TOTAL ROWS: {len(ipums_merged)}''')
print('\n')
for column in ipums_merged.columns.values:
    total = len(ipums_merged)
    nulls = ipums_merged[column].isna().sum()
    percentage = (nulls / total) * 100
    print(f'''{column}: {nulls} null values, {percentage:.2f}%''')

TOTAL ROWS: 224526


index: 0 null values, 0.00%
STATEFIP: 0 null values, 0.00%
COUNTYFIP: 0 null values, 0.00%
INDNAICS: 0 null values, 0.00%
PERWT: 0 null values, 0.00%
INCWAGE: 0 null values, 0.00%
2018 Onward ACS/PRCS INDNAICS CODE: 0 null values, 0.00%
Industry Title: 0 null values, 0.00%


In [29]:
for column in ipums_merged.columns.values:
    print(f'''{column}: {len(ipums_merged[column].unique())} unique values''')

index: 224526 unique values
STATEFIP: 1 unique values
COUNTYFIP: 35 unique values
INDNAICS: 269 unique values
PERWT: 804 unique values
INCWAGE: 889 unique values
2018 Onward ACS/PRCS INDNAICS CODE: 269 unique values
Industry Title: 269 unique values


### Merging on counties

In [30]:
ipums_w_counties = pd.merge(ipums_merged, county_info, on='COUNTYFIP', how='left')

In [31]:
print(len(ipums_merged), len(county_info), ipums_w_counties.shape)

224526 58 (224526, 20)


In [32]:
ipums_w_counties.head()

,index,STATEFIP,COUNTYFIP,INDNAICS,PERWT,INCWAGE,2018 Onward ACS/PRCS INDNAICS CODE,Industry Title,FIPS,County,State,"County, State",EDD County,Census County,Population - Households,Rural/Urban,Redstone Regions,WF Regions,CDI Regions,Population
0,156866,6,37,4853,21.0,23100,4853,Taxi and limousine service,6037.0,Los Angeles,California,"Los Angeles, California",Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570"
1,157639,6,37,4853,11.0,28000,4853,Taxi and limousine service,6037.0,Los Angeles,California,"Los Angeles, California",Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570"
2,157919,6,75,4853,4.0,1000,4853,Taxi and limousine service,6075.0,San Francisco,California,"San Francisco, California",San Francisco County,"San Francisco County, California","172,190",Urban,Bay Area,Bay Area,Bay Area,"874,961"
3,158355,6,75,4853,147.0,1000,4853,Taxi and limousine service,6075.0,San Francisco,California,"San Francisco, California",San Francisco County,"San Francisco County, California","172,190",Urban,Bay Area,Bay Area,Bay Area,"874,961"
4,158357,6,37,4853,35.0,28000,4853,Taxi and limousine service,6037.0,Los Angeles,California,"Los Angeles, California",Los Angeles County,"Los Angeles County, California","2,207,265",Urban,Los Angeles,Greater Los Angeles,Los Angeles,"10,081,570"


In [33]:
ipums_regions = ipums_w_counties[['INDNAICS',
                                          'INCWAGE',
                                          'Industry Title',
                                          'County',
                                          'CDI Regions',
                                          'PERWT']]

In [34]:
ipums_regions

,INDNAICS,INCWAGE,Industry Title,County,CDI Regions,PERWT
0,4853,23100,Taxi and limousine service,Los Angeles,Los Angeles,21.0
1,4853,28000,Taxi and limousine service,Los Angeles,Los Angeles,11.0
2,4853,1000,Taxi and limousine service,San Francisco,Bay Area,4.0
3,4853,1000,Taxi and limousine service,San Francisco,Bay Area,147.0
4,4853,28000,Taxi and limousine service,Los Angeles,Los Angeles,35.0
...,...,...,...,...,...,...
224521,3241m,65000,Miscellaneous petroleum and coal products,Sonoma,Bay Area,301.0
224522,3241m,9600,Miscellaneous petroleum and coal products,San Diego,San Diego-Imperial,111.0
224523,3241m,53000,Miscellaneous petroleum and coal products,Los Angeles,Los Angeles,122.0
224524,3241m,98000,Miscellaneous petroleum and coal products,Santa Clara,Bay Area,42.0


### Statewide Industry % - wage > state mean

In [35]:
ca_wt_mean_wage = \
    (sum([row['INCWAGE'] * row['PERWT'] for index, row in ipums_regions.iterrows()])) / \
    ipums_regions['PERWT'].sum()
ca_wt_mean_wage

49936.06306161174

Unweighted median and mean wage for comparison:

In [36]:
ca_median_wage = ipums_regions['INCWAGE'].median()
ca_median_wage

30000.0

In [37]:
ca_mean_wage = ipums_regions['INCWAGE'].mean()
ca_mean_wage

52498.9479614833

In [38]:
# same code because INCWAGE accounts for PERWT
ipums_regions['Above CA Mean'] = ipums_regions['INCWAGE'] > ca_wt_mean_wage

In [39]:
ipums_regions['wt_ind_counts'] = ipums_regions['PERWT'].groupby(ipums_regions['Industry Title']).transform('sum')

In [40]:
# cast boolean as 1/0
ipums_regions["Above CA Mean"] = ipums_regions["Above CA Mean"].astype(int)

In [41]:
ipums_regions["wt_CA_above_mean"] = ipums_regions["Above CA Mean"] * ipums_regions['PERWT']

In [42]:
ipums_regions['wt_CA_high_wage_count'] = ipums_regions['wt_CA_above_mean'].groupby(
    ipums_regions['Industry Title']).transform('sum')

In [43]:
ipums_regions['wt_CA_high_wage_perc'] = (ipums_regions['wt_CA_high_wage_count'] / ipums_regions['wt_ind_counts']) * 100

In [82]:
ipums_regions['unweighted_ind_counts'] = ipums_regions['Industry Title'].groupby(ipums_regions['Industry Title']).transform('count')

In [ ]:
ipums_regions

In [93]:
high_wage_ca_wt = ipums_regions.drop_duplicates(subset='Industry Title')
high_wage_ca_wt = high_wage_ca_wt[['Industry Title', 'unweighted_ind_counts','wt_ind_counts', 'wt_CA_high_wage_count', 'wt_CA_high_wage_perc']]
high_wage_ca_wt = high_wage_ca_wt.sort_values(by='wt_CA_high_wage_perc', ascending=False)

In [94]:
high_wage_ca_wt = high_wage_ca_wt.reset_index().iloc[:,1:]

In [95]:
high_wage_ca_wt.to_feather('data/high_wage_ca_wt.fea')

In [111]:
high_wage_ca_wt.to_csv('data/high_wage_ca_wt.csv', index=False)

In [96]:
pd.set_option('display.max_rows', None)
high_wage_ca_wt

,Industry Title,unweighted_ind_counts,wt_ind_counts,wt_CA_high_wage_count,wt_CA_high_wage_perc
0,Coal mining,3,713.0,713.0,100.000000
1,Tobacco,2,62.0,55.0,88.709677
2,Software publishers,405,40383.0,34271.0,84.864918
3,"Electric and gas, and other combinations",364,31461.0,25762.0,81.885509
4,Internet publishing and broadcasting and web s...,898,84500.0,67881.0,80.332544
5,Not specified utilities,41,3260.0,2552.0,78.282209
6,Sewage treatment facilities,108,10351.0,7947.0,76.775191
7,Computer systems design and related services,6238,613483.0,469415.0,76.516383
8,"Engine, turbine, and power transmission equipm...",53,6402.0,4866.0,76.007498
9,"Other information services, except libraries a...",74,6872.0,5101.0,74.228754


### Breakdown by Regions Pipeline

In [97]:
regions = ipums_regions['CDI Regions'].unique()
regions

array(['Los Angeles', 'Bay Area', 'Central Coast', 'Central Valley',
       'Inland Empire', 'Orange', 'Sacramento', 'San Diego-Imperial',
       'Shasta / Cascades', nan, 'Redwood Coast'], dtype=object)

In [98]:
df_LA = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[0]].copy()
df_Bay_Area = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[1]].copy()
df_Cent_Coast = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[2]].copy()
df_Cent_Valley = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[3]].copy()
df_IE = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[4]].copy()
df_OC = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[5]].copy()
df_Sac = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[6]].copy()
df_SD = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[7]].copy()
df_Shasta = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[8]].copy()
df_nan = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[9]].copy()
df_Redwood_Coast = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[10]].copy()

In [99]:
region_dfs = [df_LA, df_Bay_Area, df_Cent_Coast, df_Cent_Valley, df_IE, df_OC,
              df_Sac, df_SD, df_Shasta, df_nan, df_Redwood_Coast]

In [100]:
def add_to_region_df(df):
    df['region_wt_mean_wage'] = \
        (sum([row['INCWAGE'] * row['PERWT'] for index, row in df.iterrows()])) / df['PERWT'].sum()
    df['above_region_mean'] = df['INCWAGE'] > df['region_wt_mean_wage']
    df['wt_reg_ind_counts'] = df['PERWT'].groupby(df['Industry Title']).transform('sum')
    df["above_region_mean"] = df["above_region_mean"].astype(int)
    df["wt_reg_above_mean"] = df["above_region_mean"] * df['PERWT']
    df['wt_reg_high_wage_count'] = df['wt_reg_above_mean'].groupby(
        df['Industry Title']).transform('sum')
    df['wt_reg_high_wage_perc'] = (df['wt_reg_high_wage_count'] / df['wt_reg_ind_counts']) * 100
    return df

In [101]:
for df in region_dfs:
    df = add_to_region_df(df)

### Region Industry % - wage > region mean

In [102]:
def get_region_high_wage(df, new_df):
    new_df = df.drop_duplicates(subset='Industry Title')
    new_df = new_df[['CDI Regions', 'Industry Title', 'region_wt_mean_wage', 'unweighted_ind_counts',
                     'wt_reg_ind_counts', 'wt_reg_high_wage_count', 'wt_reg_high_wage_perc']]
    new_df = new_df.sort_values(by='wt_reg_high_wage_perc', ascending=False)
    new_df = new_df.reset_index().iloc[:,1:]
    return new_df

In [103]:
# initialize new dfs
df_LA_high_wage = df_LA.copy()
df_Bay_Area_high_wage = df_Bay_Area.copy()
df_Cent_Coast_high_wage = df_Cent_Coast.copy()
df_Cent_Valley_high_wage = df_Cent_Valley.copy()
df_IE_high_wage = df_IE.copy()
df_OC_high_wage = df_OC.copy()
df_Sac_high_wage = df_Sac.copy()
df_SD_high_wage = df_SD.copy()
df_Shasta_high_wage = df_Shasta.copy()
df_nan_high_wage = df_nan.copy()
df_Redwood_Coast_high_wage = df_Redwood_Coast.copy()

In [104]:
region_high_wage_dfs = [df_LA_high_wage, df_Bay_Area_high_wage, df_Cent_Coast_high_wage, 
                        df_Cent_Valley_high_wage, df_IE_high_wage, df_OC_high_wage,
              df_Sac_high_wage, df_SD_high_wage, df_Shasta_high_wage, df_nan_high_wage, 
                        df_Redwood_Coast_high_wage]

In [105]:
for i in range(len(region_dfs)):
    region_high_wage_dfs[i] = get_region_high_wage(region_dfs[i], region_dfs[i].copy())

In [106]:
regions_high_wage_merged = pd.concat(region_high_wage_dfs)

In [70]:
regions_high_wage_merged.reset_index().iloc[:,1:].to_feather('data/regions_high_wage_merged.fea')

In [109]:
regions_high_wage_merged.reset_index().iloc[:,1:].to_csv('data/regions_high_wage_merged.csv', index=False)

In [71]:
regions_high_wage_merged = regions_high_wage_merged.groupby(['CDI Regions','Industry Title']).agg(lambda x: x)

In [72]:
regions_high_wage_merged = regions_high_wage_merged.sort_values(['CDI Regions', 'wt_reg_high_wage_perc'], ascending=(True, False))

In [73]:
pd.set_option('display.max_rows', None)
regions_high_wage_merged

region_wt_mean_wage  \
CDI Regions        Industry Title                                                            
Bay Area           Engine, turbine, and power transmission equipme...         71590.056343   
                   Metal ore mining                                           71590.056343   
                   Nonferrous metal, except aluminum, production a...         71590.056343   
                   Not specified metal industries                             71590.056343   
                   Tires                                                      71590.056343   
                   Veneer, plywood, and engineered wood products              71590.056343   
                   Nonmetallic mineral mining and quarrying                   71590.056343   
                   Forestry, except logging                                   71590.056343   
                   Internet publishing and broadcasting and web se...         71590.056343   
                   Metal forgings and stampings                               71590.056343   
                   Not specified utilities                                    71590.056343   
                   Software publishers                                        71590.056343   
                   Paint, coating, and adhesives                              71590.056343   
                   Computer systems design and related services               71590.056343   
                   Computer and peripheral equipment manufacturing            71590.056343   
                   Commercial, industrial, and other intangible as...         71590.056343   
                   Pharmaceuticals and medicines                              71590.056343   
                   Communications, audio, and video equipment                 71590.056343   
                   Farm supplies merchant wholesalers                         71590.056343   
                   Footwear Manufacturing                                     71590.056343   
                   Agricultural implements                                    71590.056343   
                   Data processing, hosting, and related services             71590.056343   
                   Aerospace products and parts manufacturing                 71590.056343   
                   Other information services, except libraries an...         71590.056343   
                   Prefabricated wood buildings and mobile homes              71590.056343   
                   Management of companies and enterprises                    71590.056343   
                   Petroleum refining                                         71590.056343   
                   Other transportation equipment manufacturing               71590.056343   
                   Nondepository credit and related activities                71590.056343   
                   Scientific research and development services               71590.056343   
                   Labor unions                                               71590.056343   
                   Oil and gas extraction                                     71590.056343   
                   Securities, commodities, funds, trusts, and oth...         71590.056343   
                   Electric and gas, and other combinations                   71590.056343   
                   Sewage treatment facilities                                71590.056343   
                   Electronic components and products, n.e.c.                 71590.056343   
                   Miscellaneous petroleum and coal products                  71590.056343   
                   Navigational, measuring, electromedical, and co...         71590.056343   
                   Aircraft and parts manufacturing                           71590.056343   
                   Water, steam, air conditioning, and irrigation ...         71590.056343   
                   Telecommunications, except wired telecommunicat...         71590.056343   
                   Electric p